In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pickle
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv("prep.csv",index_col=None)
df2=data

In [21]:
df2 = pd.get_dummies(df2, drop_first=True)
df2

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [25]:
df2['dm_yes'].value_counts()

0    263
1    136
Name: dm_yes, dtype: int64

In [4]:
indep_X=df2.drop('classification_yes', 1)
dep_Y=df2['classification_yes']

In [5]:
# Recrusive Feature Elimination process
def rfeFeature(indep_X,dep_Y,n):
        rfelist=[]
        
        DT= DecisionTreeClassifier(criterion = 'gini', max_features='sqrt',splitter='best',random_state = 0)
        
        rfemodellist=[DT] 
        
        for i in  rfemodellist:
            
            rfe = RFE(i, n)
            rfe_fit = rfe.fit(indep_X, dep_Y)
            rfe_feature=rfe_fit.transform(indep_X)
            rfelist.append(rfe_feature)
            
            selected_features=indep_X.columns[rfe_fit.support_]
            print("Selected Features",selected_features)
        return rfelist

In [6]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)        
        return X_train, X_test, y_train, y_test,sc

In [7]:
def cm_prediction(classifier,X_test):
        y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test, y_pred)
        
        from sklearn.metrics import accuracy_score 
        from sklearn.metrics import classification_report 
        #from sklearn.metrics import confusion_matrix
        #cm = confusion_matrix(y_test, y_pred)
        
        Accuracy=accuracy_score(y_test, y_pred)
        
        report=classification_report(y_test, y_pred)
        return  classifier,Accuracy

In [8]:
def Decision(X_train,y_train,X_test):
        
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        #classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  cm_prediction(classifier,X_test)

In [9]:
def rfe_classification(accdes): 
    
    rfedataframe=pd.DataFrame(index=['DecisionTree'],columns=['Decision'])
    rfedataframe['Decision']=accdes
       
    return rfedataframe

In [10]:
accdes=[]

In [11]:
rfelist=rfeFeature(indep_X,dep_Y,4) 

Selected Features Index(['hrmo', 'sg_c', 'rbc_normal', 'dm_yes'], dtype='object')


In [13]:
for i in rfelist:   
    
    X_train, X_test, y_train, y_test,sc=split_scalar(i,dep_Y) 
    classifier,Accuracy=Decision(X_train,y_train,X_test)  
    accdes.append(Accuracy)
    
result=rfe_classification(accdes)

In [14]:
result

,Decision
DecisionTree,0.98


In [15]:
import pickle

In [16]:
filename = 'Best_RFE_classification_Model.sav'

In [17]:
pickle.dump(classifier,open(filename,'wb'))

In [18]:
filenamesc = 'sc.pkl'

In [19]:
pickle.dump(sc,open(filenamesc,'wb'))

In [20]:
sc = pickle.load(open('sc.pkl','rb'))

In [26]:
user_input = sc.transform([[12.0,0,1,1]])
user_input

array([[-0.17550919, -0.52186246,  0.38729833,  1.36930639]])

In [27]:
loaded_model = pickle.load(open('Best_RFE_classification_Model.sav','rb'))
Prediction = loaded_model.predict(user_input)

In [29]:
print("Predicted Output :",Prediction)

Predicted Output [1]
